In [ ]:
import casadi
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation

# System parameters
m1, m2 = 1.0, 1.0  # masses
l1, l2 = 1.0, 1.0  # lengths
g = 9.81

# Define optimization variables
T = 5.0  # Time horizon
N = 300  # Number of control intervals
dt = T / N

# define dynamics
nq = 2  # two joint angles
nv = 2  # two angular velocities
nu = 1
cq = casadi.SX.sym("q", nq)  # [θ1, θ2]
cv = casadi.SX.sym("v", nv)  # [ω1, ω2]
cu = casadi.SX.sym("u", nu)  # [τ2]

# Double pendulum dynamics (derived from Lagrangian)
q1, q2 = cq[0], cq[1]
v1, v2 = cv[0], cv[1]
u2 = cu[0]

# Mass matrix elements
M11 = (m1 + m2) * l1**2 + m2 * l2**2 + 2 * m2 * l1 * l2 * casadi.cos(q2)
M12 = m2 * l2**2 + m2 * l1 * l2 * casadi.cos(q2)
M21 = M12
M22 = m2 * l2**2

# Coriolis and centrifugal terms
C1 = -m2 * l1 * l2 * casadi.sin(q2) * (2 * v1 * v2 + v2**2)
C2 = m2 * l1 * l2 * casadi.sin(q2) * v1**2

# Gravity terms
G1 = (m1 + m2) * g * l1 * casadi.sin(q1) + m2 * g * l2 * casadi.sin(q1 + q2)
G2 = m2 * g * l2 * casadi.sin(q1 + q2)

# Inverse mass matrix
det = M11 * M22 - M12 * M21
invM11 = M22 / det
invM12 = -M12 / det
invM21 = -M21 / det
invM22 = M11 / det

# Acceleration equations
acc1 = invM11 * (-C1 - G1) + invM12 * (u2 - C2 - G2)
acc2 = invM21 * (-C1 - G1) + invM22 * (u2 - C2 - G2)

aba_fn = casadi.Function("aba_fn", [cq, cv, cu], [casadi.vertcat(acc1, acc2)])


# Euler integrator
def euler_integrate(q, v, u):
    q_next = q + v * dt
    v_next = v + aba_fn(q, v, u) * dt
    return q_next, v_next


# Set up optimization
opti = casadi.Opti()

Q = opti.variable(nq, N + 1)
V = opti.variable(nv, N + 1)
U = opti.variable(nu, N)

# Set initial state (hanging down)
opti.subject_to(Q[:, 0] == np.array([0, 0]))
opti.subject_to(V[:, 0] == np.array([0, 0]))

# Set final state (both links up)
opti.subject_to(Q[:, N] == np.array([np.pi, 0]))  # first link up, second straight
opti.subject_to(V[:, N] == np.array([0, 0]))

# Set dynamics constraints
for k in range(N):
    q_next, v_next = euler_integrate(Q[:, k], V[:, k], U[k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

limit = 10
for k in range(N):
    opti.subject_to(U[k] <= limit)
    opti.subject_to(U[k] >= -limit)

# Define objective
obj = 0
for k in range(N):
    obj += U[:, k].T @ U[:, k]  # control effort
    obj += (Q[:, k] - np.array([np.pi, 0])).T @ (Q[:, k] - np.array([np.pi, 0]))  # tracking error

opti.minimize(obj)

# Set solver options
p_opts = {"expand": True}
s_opts = {"max_iter": 1000}
opti.solver("ipopt", p_opts, s_opts)

# Solve
sol = opti.solve()

# Extract solution
Q_traj = sol.value(Q)
V_traj = sol.value(V)

# Animation
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)
ax.set_aspect("equal")
ax.grid(True)

(line,) = ax.plot([], [], "b-", linewidth=2)
(m1_point,) = ax.plot([], [], "bo", markersize=10)
(m2_point,) = ax.plot([], [], "ro", markersize=10)


def update(frame):
    q1, q2 = Q_traj[:, frame]

    # Calculate positions
    x1 = l1 * np.sin(q1)
    y1 = -l1 * np.cos(q1)
    x2 = x1 + l2 * np.sin(q1 + q2)
    y2 = y1 - l2 * np.cos(q1 + q2)

    line.set_data([0, x1, x2], [0, y1, y2])
    m1_point.set_data([x1], [y1])
    m2_point.set_data([x2], [y2])

    return line, m1_point, m2_point


anim = FuncAnimation(fig, update, frames=N + 1, interval=dt * 1000, blit=True)
plt.close()

html = HTML(anim.to_jshtml())
display(html)

In [ ]:
fontsize = 15
save_fig = True

# Create phase portrait
fig, ax = plt.subplots(figsize=(12, 5))

# Plot theta1 vs theta2 trajectory
ax.plot(Q_traj[0, :], Q_traj[1, :], "b-", label="Phase Portrait", linewidth=3)

# Plot start and end points
ax.plot(Q_traj[0, 0], Q_traj[1, 0], "go", label="Start (t=0s)", markersize=10)
ax.plot(Q_traj[0, -1], Q_traj[1, -1], "ro", label="End (t=5s)", markersize=10)

ax.set_xlabel("θ1 (rad)", fontsize=fontsize)
ax.set_ylabel("θ2 (rad)", fontsize=fontsize)
ax.set_title("Phase Portrait - θ1 vs θ2", fontsize=fontsize)
ax.grid(True, which="both", linestyle="-", linewidth=0.5)
ax.minorticks_on()
ax.grid(True, which="minor", linestyle=":", linewidth=0.5)
ax.legend(fontsize=fontsize)
ax.tick_params(labelsize=fontsize)

# Set x-axis ticks from -pi/2 to 1.5pi
xticks = [-np.pi / 2, 0, np.pi / 2, np.pi, 1.5 * np.pi]
xticklabels = ["-π/2", "0", "π/2", "π", "3π/2"]
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels)
ax.set_xlim(-np.pi / 2, 1.1 * np.pi)

# Set y-axis ticks from -pi to pi
yticks = [-np.pi, -np.pi / 2, 0, np.pi / 2, np.pi]
yticklabels = ["-π", "-π/2", "0", "π/2", "π"]
ax.set_ylim(-np.pi, np.pi)
ax.set_yticks(yticks)
ax.set_yticklabels(yticklabels)

plt.tight_layout()

if save_fig:
    plt.savefig("doublependulum_phase_portrait.png", dpi=300)

plt.show()